In [97]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import psycopg2

In [5]:
# Getting all data from the source file

files = [file for file in os.listdir('./data')]

In [94]:
# Function to convert datetime

def to_datetime(df, columns):
    for col in columns:
        try:
            df[col] = pd.to_datetime(df[col], format='mixed')
        except :
            pass
            

In [124]:
data_path = os.getcwd()+'\\data\\'

#storing dataframe as file_name : data
df = {}

# Param to connect to database
credentials = {
    'database':'E-commerce_analysis',
    'user':'postgres',
    'password': 'chan.chano123099',
    'host': '127.0.0.1',
    'port': 5000
}



for file in files:
    df[file] = pd.read_csv(data_path+file)
    data = df[file]
    
    
    # Remove '.csv' for db table name
    table_name = file.split('.')[0].replace(' ', '_')
    
    # Clean and transform the data
    # clean column name
    data.columns = [col.lower().replace(' ', '_').replace('-','') for col in data.columns]
    data.dropna(inplace=True)
    columns = ['order_date', 'month_of_order_date']
    to_datetime(data, columns)
    
    
    # Map the pandas dtype to convert into sql dtype
    replacement = {
        'object':'varchar',
        'datetime64[ns]':'timestamp',
        'float64':'float',
        'int64':'int'
    }
    
    # Table constraint
    col_sql = ",".join("{} {}".format(c,d) for (c,d) in zip(data.columns, data.dtypes.replace(replacement)))
    
    # Add to database
    conn = psycopg2.connect(**credentials)
    cur = conn.cursor()
    
    # Create table
    cur.execute("DROP TABLE IF EXISTS {0}".format(table_name))
    cur.execute("CREATE TABLE IF NOT EXISTS {0} ({1})".format(table_name, col_sql))
    
    data.to_csv(file, header=data.columns, index=False)
    
    my_file = open(file)
    
    sql_statement = f"""
        COPY {table_name} FROM STDIN WITH
            HEADER
            CSV
            DELIMITER AS ','
    """
    
    cur.copy_expert(sql=sql_statement,  file=my_file)
    
    print("Data successfully added to database!")
    
    conn.commit()
    conn.close()

Data successfully added to database!
Data successfully added to database!
Data successfully added to database!


In [95]:
# Let's see if the null values will be drop or need to analyse

handler = {}
for i, file in enumerate(files):
    handler[i] = pd.read_csv(data_path+file)
    
    columns = ['Order Date', 'Month of Order Date']
    
    to_datetime(handler[i], columns)
    
    print("Dtypes: ", handler[i].dtypes, "\n")
    
    #print(f"{file}: ", handler[i].isna().sum())
    #print("---------------------------------------")
    #print(f"{file}: Duplicated values: ", handler[i].duplicated().sum())
    #print("---------------------------------------")
    
    # Conclusion: Drop the null
    

Dtypes:  Order ID                object
Order Date      datetime64[ns]
CustomerName            object
State                   object
City                    object
dtype: object 

Dtypes:  Order ID         object
Amount          float64
Profit          float64
Quantity          int64
Category         object
Sub-Category     object
dtype: object 

Dtypes:  Month of Order Date     object
Category                object
Target                 float64
dtype: object 

